In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import random
import math
%matplotlib inline
from shapely.geometry import LineString
#For interactive

import geopandas as gpd
import contextily as cx

movShap = gpd.read_file(r"New/ShapeMatrice OD2016 - Passeggeri - Zone interne")
movShap_wm = movShap.to_crs(epsg=3857)

mov = pd.read_csv("New/matrice_od_2020_passeggeri/matrice_od_2020_passeggeri.csv")#Heavy



## Prepare everything

In [ ]:
timeSlots = ['00:00-00:59', '01:00-01:59', '02:00-02:59', '03:00-03:59',
       '04:00-04:59', '05:00-05:59', '06:00-06:59', '07:00-07:59',
       '08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59',
       '12:00-12:59', '13:00-13:59', '14:00-14:59', '15:00-15:59',
       '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59',
       '20:00-20:59', '21:00-21:59', '22:00-22:59', '23:00-23:59']
milanZones = ['','MILANO 1', 'MILANO 2', 'MILANO 3', 'MILANO 4', 'MILANO 5',
       'MILANO 6', 'MILANO 7', 'MILANO 8', 'MILANO 9', 'MILANO 10',
       'MILANO 11', 'MILANO 12', 'MILANO 13', 'MILANO 14', 'MILANO 15',
       'MILANO 16']
movTypes = ['COND', 'PAX', 'FERRO', 'GOMMA', 'MOTO', 'BICI', 'PIEDI', 'ALTRO']
movReason = ["STU","LAV","OCC","AFF","RIT"]
#IN Milano 13 ci sono quartieri xyz, vedo distribuzione popolazione in questi quartieri
munMov = [
    [],
    [1],#Milano 1
    [1],#Milano 2
    [2,9],#Milano 3
    [3,4],#Milano 4
    [2,9],#Milano 5
    [3],#Milano 6
    [5],#Milano 7
    [4],#Milano 8
    [4,5],#Milano 9
    [5],#Milano 10
    [6],#Milano 11
    [6,7],#Milano 12
    [6,7],#Milano 13
    [8],#Milano 14
    [8],#Milano 15
    [9]#Milano 16
]
# nil = gpd.read_file(r'New/ds964_quartieri.geojson',geometry="Location")
# nil = nil.set_crs('epsg:4326')
# nil = nil.to_crs(epsg=3857)
# nil = pd.merge(nil, pd.read_csv('New/ds964_nil.csv',sep = ";"),how='inner',left_on=['ID_NIL'],right_on=['ID_NIL'],suffixes=('', '_y'))
# nil.drop(nil.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
# nil = pd.merge(nil, nil.groupby(['Municipio'], as_index = False).sum()[["Municipio","Popolazione"]],how='inner',left_on=['Municipio'],right_on=['Municipio'],suffixes=('', 'Tot'))

In [ ]:
popFam = pd.read_csv("new/popolazioneFamiglieEtàZoneMilano.csv")
# popFam.loc[(popFam["Municipio"]== 9) & (popFam["Classe_eta_capofamiglia"]== "80 anni e più") & (popFam["Genere_capofamiglia"]== "Femmine")]
popFam  = popFam.groupby(['Municipio','Classe_eta_capofamiglia','Genere_capofamiglia','Numero_componenti','Tipologia_familiare'], as_index=True).agg(
    {'Frq':'sum'}).reset_index()
#Since have only householder in popFam, need to have data also on kids/teen
popZon = pd.read_csv("new/ds29_pop_sto_zona.csv", sep = ";")
popZon = popZon.loc[popZon["Anno"]== 2020]
# for col in popZon.columns:
#     print(popZon[col].unique())
try:
    popZon['Eta'] = popZon['Eta'].map(lambda x: x.rstrip('  e più'))
except:
    print("Already done")
    
popZon['Eta'] = popZon['Eta'].astype(int)
popZon = popZon.loc[popZon["Eta"] < 18]#Keep only part missing from other table
bins= [0,3,13,18]
labels = ['0-2 anni','3-12 anni','13-17 anni']
popZon['Classe_eta_capofamiglia'] = pd.cut(popZon['Eta'], bins=bins, labels=labels, right=False) ##Create age groups

popZon = popZon.groupby(['Municipio','Classe_eta_capofamiglia','Genere'], as_index=True).agg({'Residenti':'sum'}).reset_index()
popZon["Tipologia_familiare"] = "Single"#For consistency
popZon["Frq"] = popZon["Residenti"]
popZon["Numero_componenti"] = "1"
popZon = popZon.drop(["Residenti"],axis = 1)
popZon["Genere_capofamiglia"] = popZon["Genere"]
popZon = popZon.drop(["Genere"],axis = 1)
popMilano = pd.concat([popFam, popZon])
for i in range (1,10):
    popMilano.loc[popMilano["Municipio"] ==i,"Distribution"] =  popMilano.loc[popMilano["Municipio"] ==i,"Frq"]/popMilano.loc[popMilano["Municipio"] ==i,"Frq"].sum()

## Read ds

In [ ]:
movShap_wm_Mi = movShap_wm.loc[movShap_wm["desc_zona"].str.contains("MILANO")]#Can be changed

ax = movShap_wm_Mi.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax, source="MilanSmall.tif")
plt.axis('off')
movShap_wm_Mi.apply(lambda x: ax.annotate(text=x['desc_zona'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=20), axis=1);

In [ ]:
stores = [
    "Food_B1_StoreFront_data",
    "Food_P1_StoreFront_data",
    "Retail_C1_StoreFront_data",
    "Retail_C1_WalkIn_data",
    "Retail_C2_StoreFront_data",
    "Retail_C3_StoreFront_data",
    "Retail_C3_WalkIn_data",
    "Retail_C4_StoreFront_data",
    "Retail_C5_StoreFront_data",
    "Retail_C6_StoreFront_data",
    "Retail_C6_WalkIn_data"
]
interestedSlotsPossible = [
    ['12:00-12:59', '13:00-13:59', '14:00-14:59', '19:00-19:59',
       '20:00-20:59', '21:00-21:59', '22:00-22:59'],
    ['08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59',
       '12:00-12:59', '13:00-13:59', '14:00-14:59', '15:00-15:59',
       '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59',
       '20:00-20:59'],
]
zonaStore = "MILANO 3"


In [ ]:
selected = 8
interestedSlot = 1

## Choose dataset

In [ ]:
ds1 = pd.read_excel("New/rdatibs4pertesisimonebellini/"+stores[selected]+".xlsx").dropna().drop(0,axis=0).reset_index().drop(["index"],axis=1)
currMov = mov.loc[mov["ZONA_DEST"] == zonaStore]
interestedSlots = interestedSlotsPossible[interestedSlot]

#Remove columns of all zeros
for col in ds1.columns:
    if np.array_equal(ds1[col].values,np.zeros(len(ds1))):
        ds1.drop(col,axis=1, inplace=True)

In [ ]:
totMovTimeSlot = []
for ts in timeSlots:
    totMovTimeSlot.append(currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].sum()) #Total movements in that timeslot

#### Total movements

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(timeSlots,totMovTimeSlot)
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by time slot in " + zonaStore)
plt.show()

#### Where are movements from

In [ ]:
#Get where are from
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
whereFrom = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts]#Get all movements for ts
    currGroup = movdf.groupby(['PROV_ORIG'], as_index=False).sum()
    whereFromTs = []
    for pog in currGroup.PROV_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["PROV_ORIG"]== pog].sum()[1:].sum())#Get all movements from province and save
    whereFrom.append(whereFromTs)
labels = currGroup.PROV_ORIG

# Import Python cycling library
from itertools import cycle

# Create a colour code cycler e.g. 'C0', 'C1', etc.
color_codes = map('C{}'.format, cycle(range(len(whereFromTs))))

cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle('color', [cm(1.*i/len(whereFromTs)) for i in range(len(whereFromTs))])


for i in range(0,len(whereFromTs)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*whereFrom))[i]),label=labels.values[i])

        bot = np.array(list(list(zip(*whereFrom))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*whereFrom))[i]),label=labels.values[i],
               bottom=bot)
        bot += np.array(list(list(zip(*whereFrom))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(1.2,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by origin in " + zonaStore)
plt.show()


In [ ]:
import plotly.express as px
origindf = pd.DataFrame(list(list(zip(*whereFrom))), columns =timeSlots).T
origindf.columns = labels
fig = px.bar(origindf, x=timeSlots,y = origindf.columns, title="Movements by origin in " + zonaStore)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

fig.show()

#### More in depth on Milan

In [ ]:
#Get where are from
totMovTimeSlotMilan = []
whereFromMilan = []
for ts in timeSlots:
    movdf = currMov.loc[(currMov["FASCIA_ORARIA"] == ts) &(currMov["PROV_ORIG"] == "MI")]#Get all movements for ts
    currGroup = movdf.groupby(['ZONA_ORIG'], as_index=False).sum()
    whereFromTs = []
    for zog in currGroup.ZONA_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["ZONA_ORIG"]== zog].sum()[1:].sum())#Get all movements from province and save
    whereFromMilan.append(whereFromTs)
    totMovTimeSlotMilan.append(sum(whereFromTs))
labelsMilan = currGroup.ZONA_ORIG


origindf = pd.DataFrame(list(list(zip(*whereFromMilan))), columns =timeSlots).T
origindf.columns = labelsMilan
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    fig = px.bar(origindf, x=timeSlots,y = origindf.columns, 
             title="Movements by origin in " + zonaStore + "(Milan province only)")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

fig.show()

In [ ]:
#Get where are from
totMovTimeSlotMilanCity = []
whereFromMilanCity = []
for ts in timeSlots:
    movdf = currMov.loc[(currMov["FASCIA_ORARIA"] == ts) &(currMov["ZONA_ORIG"].str.contains("MILANO"))]#Get all movements for ts
    currGroup = movdf.groupby(['ZONA_ORIG'], as_index=False).sum()
    whereFromTs = []
    for zog in currGroup.ZONA_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["ZONA_ORIG"]== zog].sum()[1:].sum())#Get all movements from province and save
    whereFromMilanCity.append(whereFromTs)
    totMovTimeSlotMilanCity.append(sum(whereFromTs))

labelsMilanCity = currGroup.ZONA_ORIG


origindf = pd.DataFrame(list(list(zip(*whereFromMilanCity))), columns =timeSlots).T
origindf.columns = labelsMilanCity
fig = px.bar(origindf.copy(), x=timeSlots,y = origindf.copy().columns, 
             title="Movements by origin in " + zonaStore + "(Milan city only)")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

fig.show()


#### How people arrived there

In [ ]:
#Get how they arrived there
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
howArrive = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].to_frame().T#Get all movements for ts
    howArrivets = []
    for typ in movTypes:#For each movement type (CAR, bike etc)
        filter_col = [col for col in mov if col.endswith(typ)]
        howArrivets.append(movdf[filter_col].sum().sum())
    howArrive.append(howArrivets)
    
for i in range(0,len(movTypes)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movTypes[i])
        bot = np.array(list(list(zip(*howArrive))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movTypes[i],
               bottom=bot)
        bot += np.array(list(list(zip(*howArrive))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by type in " + zonaStore)
plt.show()


#### Why people arrived there

In [ ]:
#Get why they arrived there
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
whyArrive = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].to_frame().T#Get all movements for ts
    whyArrivets = []
    for typ in movReason:#For each movement type (CAR, bike etc)
        filter_col = [col for col in mov if col.startswith(typ)]
        whyArrivets.append(movdf[filter_col].sum().sum())
    whyArrive.append(whyArrivets)
    
for i in range(0,len(movReason)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*whyArrive))[i]),label=movReason[i])
        bot = np.array(list(list(zip(*whyArrive))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*whyArrive))[i]),label=movReason[i],
               bottom=bot)
        bot += np.array(list(list(zip(*whyArrive))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by reason in " + zonaStore)
plt.show()


### Apply analysis to store movements

### Where are the people outside store from

N.B. The data is statistical and can be different from reality especially with low values (outside of rush hours)

In [ ]:
ds1

In [ ]:
ds1Where = ds1.copy()
whereData = (np.asarray(whereFrom).T/totMovTimeSlot).T
whereucAll = []
ch = []
for col in ds1Where.columns:
    if col != "Date":
        for i in range(0,len(totMovTimeSlot)):
            choices = np.random.choice(labels, ds1Where[col][i], p=whereData[i])
            (unique, counts) = np.unique(choices, return_counts=True)
            ch.append((unique, counts))
#             unique = unique[counts>2]#CAN BE ADJUSTED
#             counts = counts[counts>2]#
            whereucAll.append((unique, counts))
            ds1Where[col][i]=np.char.add(np.char.add(unique.astype(str),":"), counts.astype(str))
ds1Where.loc[ds1Where["Date"].isin(interestedSlots)]

##### See where they come from in Milan and estimate type of people

In [ ]:
whereDataMilan = (np.asarray(whereFromMilan).T/totMovTimeSlotMilan).T
chMil = []
whereucMi = []
count = 0
for col in ds1Where.columns:
    if col != "Date": 
        for i in range(0,len(totMovTimeSlotMilan)):
            if np.where(ch[i + count*24][0] == "MI")[0].size != 0: #If there are people moving from Milan
                choices = np.random.choice(labelsMilan, 
                                           ch[i + count*24][1][np.where(ch[i + count*24][0] == "MI")[0]], 
                                           p=whereDataMilan[i])
                (unique, counts) = np.unique(choices, return_counts=True)
                whereucMi.append((unique, counts))
#                 unique = unique[counts>2]#CAN BE ADJUSTED
#                 counts = counts[counts>2]#
                chMil.append((unique, counts,col,i))
                ds1Where[col][i]=np.concatenate(
                    (np.delete(ds1Where[col][i],np.flatnonzero(np.core.defchararray.find(ds1Where[col][i],"MI")!=-1)),#Remove MI
                    np.char.add(np.char.add(unique.astype(str),":"), counts.astype(str))))#Add more specific
            else:
                whereucMi.append((0,0))
                
        count+=1

ds1Where.loc[ds1Where["Date"].isin(interestedSlots)]

#### See what kind of person

In [ ]:
import re
from itertools import repeat

ds1Who = ds1.copy()
whouc =list(repeat((0,0), len(whereucMi)))

for cell in chMil:#Get movements from Milan city
    choices =  np.array([])
    for i in np.argwhere(np.in1d(cell[0], milanZones)).ravel():#For each movement
        #Get municipi in interested zone 
        #For each see population
        for n in munMov[int(re.search(r'\d+', cell[0][i]).group())]:
            #Draw
            choice = np.random.choice(popMilano.loc[popMilano["Municipio"] == n]["Classe_eta_capofamiglia"], 
                                       cell[1][i]-int(cell[1][i]/len(munMov[int(re.search(r'\d+', cell[0][i]).group())])), 
                                       p=popMilano.loc[popMilano["Municipio"] == n]["Distribution"])
            #Save
            choices = np.concatenate((choices,choice))
    #Group together
    (u, c) = np.unique(choices, return_counts=True)
    whouc[cell[3]+(ds1.columns.get_loc(cell[2])-1)*24] = (u, c)
    ds1Who[cell[2]][cell[3]]=np.char.add(np.char.add(u.astype(str),":"), c.astype(str))
ds1Who.loc[ds1Who["Date"].isin(interestedSlots)]

Data driven --> tipi di famiglia

### How does the people arrive there

In [ ]:
ds1How = ds1.copy()
howuc = []
howData = (np.asarray(howArrive).T/totMovTimeSlot).T
for col in ds1How.columns:
    if col != "Date":
        for i in range(0,len(totMovTimeSlot)):
            choices = np.random.choice(movTypes, ds1How[col][i], p=howData[i])
            (unique, counts) = np.unique(choices, return_counts=True)
#             unique = unique[counts>5]#CAN BE ADJUSTED
#             counts = counts[counts>5]#
            howuc.append((unique, counts))
            ds1How[col][i]=np.char.add(np.char.add(unique.astype(str),":"), counts.astype(str))
ds1How.loc[ds1How["Date"].isin(interestedSlots)]

Data driven--> percheggio e necessità trasporti

### Why does the people arrive there

In [ ]:
ds1Why = ds1.copy()
whyuc= []
whyData = (np.asarray(whyArrive).T/totMovTimeSlot).T
for col in ds1Why.columns:
    if col != "Date":
        for i in range(0,len(totMovTimeSlot)):
            choices = np.random.choice(movReason, ds1Why[col][i], p=whyData[i])
            (unique, counts) = np.unique(choices, return_counts=True)
#             unique = unique[counts>10]#CAN BE ADJUSTED
#             counts = counts[counts>10]#
            whyuc.append((unique, counts))
            ds1Why[col][i]=np.char.add(np.char.add(unique.astype(str),":"), counts.astype(str))
ds1Why.loc[ds1Why["Date"].isin(interestedSlots)]

Data driven--> offerte per tipologia o differenti bisogni

## Graphs

In [ ]:
col = random.randint(1, len(ds1)-1)


## Age groups

#### See if correct distribution through histograms

In [ ]:
currDay = whouc[col*24:(col+1)*24]

fig = plt.figure()
color = ["Blue", "Green", "Red", "Pink", "Yellow", "Purple", "Grey", "Lime", "Black"]
ax = fig.add_axes([0,0,1,1])
ceta = popMilano["Classe_eta_capofamiglia"].unique()
for r in range(0,len(ceta)):
    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == ceta[r])[0]) !=0:
            loc = np.where(currDay[i][0] == ceta[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    if r ==0:
        plt.bar(timeSlots,currReason,label=ceta[r], color = color[r])
        bot = currReason
    else:
        plt.bar(timeSlots,currReason,label=ceta[r], color = color[r],
               bottom=bot)
        bot = np.add(bot,currReason)
plt.legend(loc="upper left",bbox_to_anchor=(1.0,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by type in " + zonaStore)
plt.show()

#### 7 subplots for each age group

In [ ]:
plt.figure(figsize=(16, 16)) 
plt.subplots_adjust(top=1.9,
                    bottom=0.1)
currTot = []
for r in range(0,len(ceta)):
    plt.subplot(4, 2, r+1)

    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == ceta[r])[0]) !=0:
            loc = np.where(currDay[i][0] == ceta[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    plt.bar(timeSlots,currReason,label=ceta[r], color = color[r])
    plt.title(ceta[r])
    plt.xticks(timeSlots,rotation='vertical')
    currTot.append(currReason)



#### Stacked area chart 

In [ ]:
plt.stackplot(timeSlots, currTot,labels=ceta,colors = color)
plt.title("Stacked area chart")
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper left')

In [ ]:
y = np.row_stack((currTot))
percent = y /  y.sum(axis=0).astype(float) * 100 

plt.stackplot(timeSlots, percent,labels=ceta,colors = color)
plt.title('100 % stacked area chart')
plt.ylabel('Percent (%)')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))

#### Cum distribution (for age groups)

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, np.cumsum(currTot[c])/np.sum(currTot[c]), label=ceta[c],color = color[c])
plt.ylabel('Percent (%)')
plt.title('Cumulative distribution')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, currTot[c], label=ceta[c],color = color[c])
plt.ylabel('Values')
plt.title('Line chart')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))
plt.plot(timeSlots, currTot[0])

## Reason

In [ ]:
currDay = whyuc[col*24:(col+1)*24]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

for r in range(0,len(movReason)):
    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == movReason[r])[0]) !=0:
            loc = np.where(currDay[i][0] == movReason[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    if r ==0:
        plt.bar(timeSlots,currReason,label=movReason[r],color = color[r])
        bot = currReason
    else:
        plt.bar(timeSlots,currReason,label=movReason[r],color = color[r],
               bottom=bot)
        bot = np.add(bot,currReason)
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by reason in " + zonaStore)
plt.show()

In [ ]:
plt.figure(figsize=(16, 16)) 
plt.subplots_adjust(top=1.3,
                    bottom=0.001)
currTot = []
for r in range(0,len(movReason)):
    plt.subplot(3, 2, r+1)

    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == movReason[r])[0]) !=0:
            loc = np.where(currDay[i][0] == movReason[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    plt.bar(timeSlots,currReason,label=movReason[r], color = color[r])
    plt.title(movReason[r])
    plt.xticks(timeSlots,rotation='vertical')
    currTot.append(currReason)



In [ ]:
plt.stackplot(timeSlots, currTot,
              labels=movReason,colors = color)
plt.title("Stacked area chart")
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper left')

In [ ]:
y = np.row_stack((currTot))
percent = y /  y.sum(axis=0).astype(float) * 100 

plt.stackplot(timeSlots, percent,labels=movReason,colors = color)
plt.title('100 % stacked area chart')
plt.ylabel('Percent (%)')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, np.cumsum(currTot[c])/np.sum(currTot[c]), label=movReason[c],color = color[c])
plt.ylabel('Percent (%)')
plt.title('Cumulative distribution')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.2,1.0))

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, currTot[c], label=movReason[c],color = color[c])
plt.ylabel('Values')
plt.title('Line chart')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))
plt.plot(timeSlots, currTot[0])

## How

In [ ]:
currDay = howuc[col*24:(col+1)*24]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

for r in range(0,len(movTypes)):
    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == movTypes[r])[0]) !=0:
            loc = np.where(currDay[i][0] == movTypes[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    if r ==0:
        plt.bar(timeSlots,currReason,label=movTypes[r])
        bot = currReason
    else:
        plt.bar(timeSlots,currReason,label=movTypes[r],
               bottom=bot)
        bot = np.add(bot,currReason)
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by type in " + zonaStore)
plt.show()


In [ ]:
plt.figure(figsize=(16, 16)) 
plt.subplots_adjust(top=1.9,
                    bottom=0.001)
currTot = []
for r in range(0,len(movTypes)):
    plt.subplot(4, 2, r+1)

    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == movTypes[r])[0]) !=0:
            loc = np.where(currDay[i][0] == movTypes[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    plt.bar(timeSlots,currReason,label=movTypes[r], color = color[r])
    plt.title(movTypes[r])
    plt.xticks(timeSlots,rotation='vertical')
    currTot.append(currReason)



In [ ]:
plt.stackplot(timeSlots, currTot,
              labels=movTypes,colors = color)
plt.title("Stacked area chart")
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper left')

In [ ]:
y = np.row_stack((currTot))
percent = y /  y.sum(axis=0).astype(float) * 100 

plt.stackplot(timeSlots, percent,labels=movTypes,colors = color)
plt.title('100 % stacked area chart')
plt.ylabel('Percent (%)')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, np.cumsum(currTot[c])/np.sum(currTot[c]), label=movTypes[c],color = color[c])
plt.ylabel('Percent (%)')
plt.title('Cumulative distribution')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.2,1.0))

In [ ]:
for c in range (0,len(currTot)):
    plt.plot(timeSlots, currTot[c], label=movTypes[c],color = color[c])
plt.ylabel('Values')
plt.title('Line chart')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(1.4,1.0))
plt.plot(timeSlots, currTot[0])

## From where

In [ ]:
currDay = whereucAll[col*24:(col+1)*24]

whereFrom = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts]#Get all movements for ts
    currGroup = movdf.groupby(['PROV_ORIG'], as_index=False).sum()
    whereFromTs = []
    for pog in currGroup.PROV_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["PROV_ORIG"]== pog].sum()[1:].sum())#Get all movements from province and save
    whereFrom.append(whereFromTs)

currReasonTot = []
for r in range(0,len(whereFromTs)):
    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == labels.values[r])[0]) !=0:
            loc = np.where(currDay[i][0] == labels.values[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    currReasonTot.append(currReason)

origindf = pd.DataFrame(currReasonTot, columns =timeSlots).T
origindf.columns = labels
fig = px.bar(origindf, x=timeSlots,y = origindf.columns, title="Movements by origin in " + zonaStore)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

fig.show()


In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle('color', [cm(1.*i/len(whereFromTs)) for i in range(len(whereFromTs))])
ax.stackplot(timeSlots, currReasonTot,
              labels=labels)
ax.set_title("Stacked area chart")
ax.set_xticks(timeSlots,rotation='vertical')
ax.legend(loc='upper left',bbox_to_anchor=(1.0,1.0))


In [ ]:
y = np.row_stack((currReasonTot))
percent = y /  y.sum(axis=0).astype(float) * 100 

plt.stackplot(timeSlots, percent,labels=labels,colors = color)
plt.title('100 % stacked area chart')
plt.ylabel('Percent (%)')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(2.5,1.0))

#### Chord for each place in one timeslot

In [ ]:
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data

hv.extension('bokeh')
hv.output(size=200)

links = pd.DataFrame(data['links'])
# print(links.head(3))
nodes = hv.Dataset(pd.DataFrame(data['nodes']), 'index')
print(pd.DataFrame(data['nodes']))
nodes.data.head()

df =(pd.DataFrame(currReasonTot, columns = timeSlots).T)
df = df.T
df["target"] = 0
df["source"] = df.index
df["value"] = df["12:00-12:59"] 
links = df[["source","target","value"]]
# links["value"] = links["12:00-12:59"] 
links
# df =(pd.DataFrame(currReasonTot, columns = timeSlots).T)
# df.columns = labels
# df = df.T
# nodes = hv.Dataset(df)
chord = hv.Chord(links[links["value"]>0])#.select(value=(5, None))
# # chord.opts(
# #     opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('01:00-01:59').str(), 
# #                labels='PROV_ORIG', node_color=dim('index').str()))
chord
# links[links["value"]>0]

#### Chord for each timeslot from everywhere

In [ ]:
currDay = whereucMi[col*24:(col+1)*24]

totMovTimeSlotMilan = []
whereFromMilan = []
for ts in timeSlots:
    movdf = currMov.loc[(currMov["FASCIA_ORARIA"] == ts) &(currMov["PROV_ORIG"] == "MI")]#Get all movements for ts
    currGroup = movdf.groupby(['ZONA_ORIG'], as_index=False).sum()
    whereFromTs = []
    for zog in currGroup.ZONA_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["ZONA_ORIG"]== zog].sum()[1:].sum())#Get all movements from province and save
    whereFromMilan.append(whereFromTs)
    totMovTimeSlotMilan.append(sum(whereFromTs))
labelsMilan = currGroup.ZONA_ORIG

# Create a colour code cycler e.g. 'C0', 'C1', etc.
color_codes = map('C{}'.format, cycle(range(len(whereFromTs))))

cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle('color', [cm(1.*i/len(whereFromTs)) for i in range(len(whereFromTs))])

currReasonTot = []
for r in range(0,len(whereFromTs)):
    currReason = []
    for i in range (0,len(currDay)):
        if len(np.where(currDay[i][0] == labelsMilan.values[r])[0]) !=0:
            loc = np.where(currDay[i][0] == labelsMilan.values[r])[0].item()
            currReason.append(currDay[i][1][loc])
        else:
            currReason.append(0)
    currReasonTot.append(currReason)
            
origindf = pd.DataFrame(currReasonTot, columns =timeSlots).T
origindf.columns = labelsMilan
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    fig = px.bar(origindf, x=timeSlots,y = origindf.columns, 
             title="Movements by origin in " + zonaStore + "(Milan province only)")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle('color', [cm(1.*i/len(whereFromTs)) for i in range(len(whereFromTs))])
ax.stackplot(timeSlots, currReasonTot,
              labels=labelsMilan)
ax.set_xticks(timeSlots,rotation='vertical')
ax.set_title("Stacked area chart")
ax.legend(loc='upper left',bbox_to_anchor=(1.0,1.0))


In [ ]:
y = np.row_stack((currReasonTot))
percent = y /  y.sum(axis=0).astype(float) * 100 

plt.stackplot(timeSlots, percent,labels=labelsMilan,colors = color)
plt.title('100 % stacked area chart')
plt.ylabel('Percent (%)')
plt.margins(0, 0) # Set margins to avoid "whitespace"
plt.xticks(timeSlots,rotation='vertical')
plt.legend(loc='upper right',bbox_to_anchor=(2.5,1.0))

#### Chord diagrams (for transports)

In [ ]:
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data

hv.extension('bokeh')
hv.output(size=200)

links = pd.DataFrame(data['links'])
print(links.head(3))
nodes = hv.Dataset(pd.DataFrame(data['nodes']), 'index')
print(pd.DataFrame(data['nodes']))
nodes.data.head()
chord = hv.Chord((links, nodes)).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='name', node_color=dim('index').str()))